In [1]:
import sqlite3
import numpy as np
import requests
import os
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

d:\Python\envs\ml\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Python\envs\ml\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [3]:
load_dotenv()

True

In [4]:
def cosine_similarity(a, b):
    
    if np.linalg.norm(a) == 0 or np.linalg.norm(b) == 0:
        return 0.0
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [5]:
query_text = "What are the ingredients in the recipe?"
db_path='embeddings.db' 
top_k=3
min_sim=0.5

In [6]:
query_emb = model.encode([query_text])[0]

In [7]:
conn = sqlite3.connect(db_path)
c = conn.cursor()
c.execute('SELECT chunk_id, text, embedding_vector FROM embeddings')
results = c.fetchall()
conn.close()

In [8]:
results

[('sample_doc_chunk_1',
  'Classic chocolate chip cookies recipe.',
  b'\xa0\x0e2\xbd\x86\xa9\x9b=\xb8~\xc0\xbb\x9a\xa6"=x"K\xbdbV\x89\xbd\xeecW=\xfco\x19\xbd\xb7\x96\x8b\xbd\x9f\xa0L<\xa9\x99\xdf\xbc6\x95\x81\xbb\xc63\xe1\xbd\rc\xf9\xbc#`\x15\xbd\x1d\xd1E\xbc\'\xaf\xa9<rB\xaa<\xb1\xcc\xdb<\xe6G\x0f\xbc\xffB\x8b;\xe6Aq\xbd\x87\x10\x82=\xdf\xe33=\xd6\x01s\xbd\xe9\xb0h=7\x12\x97=\xff\xa8\x17\xbc\xe0y\xf5\xbc\x14\x97\x00\xbd\xd5O\x8c<\x07pf<\xc7.\x87=\x83\xbbZ\xbdk\xd7\xf3<\xe8m\xb3\xbd\xfe\xab\x9b\xbc\xa5"\xc4<\x1d\xea\x08\xbc\xf2\x01T\xbd\x7f\xd8\xb9\xba\xb0\x7f\xf1;{\x88k<BA\x92=\xe3\xea\xa8\xbd\xca}\xe7\xbb\xb5yZ\xbcC2\x02=\xac\xdbk<g)\x87<\x0f\x91\xbe<g\x17\xb3\xbb\xca\x7f\x90\xbdA\x15)\xbb1;\x81\xbc\xc4\xd7\x95;__\x0b\xbe$\x04\xec\xbc8\xeb\x93<\xf2\x88\xa8=\xf6\x81\x93\xbd\x0e\xcdU=R\xb4\x00\xbc\xa1\x12B:w_\x01=\xd1\xe1\x1e\xbd\xe1\x86\x02\xbd\x05n\x7f<.\n<\xbc\xffu\xb4;\xb7\x9aC\xbd\xc5\xc8\xd0\xbc\xdbj\xa6<\xae\xce\x0e<\xe1\x98\x8e=k\xe0\xbb\xbc\xf1\x10\xc7<\x1f"E;\\\x11G<\xc2\x17

In [9]:
similarities = []

In [10]:
for chunk_id, text, emb_blob in results:
    stored_emb = np.frombuffer(emb_blob, dtype=np.float32)
    sim = cosine_similarity(query_emb, stored_emb)
    if sim >= min_sim:
        similarities.append((sim, chunk_id, text))

In [11]:
top_chunks = sorted(similarities, key=lambda x: x[0], reverse=True)[:top_k]

In [12]:
context = "\n".join([f"- {chunk[2]}" for chunk in top_chunks]) 
prompt = f"""You are a helpful assistant. Answer the question based ONLY on the provided context below. 
Do not use any external knowledge. If the answer is not in the context, respond exactly: "I don’t know based on the provided context."
Context:{context}
Question: {query_text}
Answer:"""

In [13]:
model_name="meta-llama/Llama-3.1-8B-Instruct:fastest"

In [14]:
API_URL = "https://router.huggingface.co/v1/chat/completions" 
headers = {"Authorization": f"Bearer {os.getenv('HF_TOKEN')}", "Content-Type": "application/json"}
messages = [{"role": "user", "content": prompt}] 
payload = {
    "model": model_name,
    "messages": messages,
    "max_tokens": 150,  
    "temperature": 0.1 
}

In [15]:
response = requests.post(API_URL, headers=headers, json=payload)
if response.status_code == 200:
    result = response.json()
    responseans =  result['choices'][0]['message']['content'].strip()  # Extract from choices
else:
    responseans = f"API Error: {response.status_code} - {response.text}"

In [16]:
if not top_chunks:
    print("No relevant chunks found. I don’t know based on the provided context.")

In [17]:
sources = [f"Chunk {chunk[1]} (sim: {chunk[0]:.2f})" for chunk in top_chunks]

In [18]:
print(f"{responseans}\n\nSources:\n" + "\n".join(sources))

The ingredients in the recipe are flour and butter.

Sources:
Chunk sample_doc_chunk_2 (sim: 0.60)
